In [11]:
from dotenv import load_dotenv
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.document_loaders import TextLoader
from langchain_openai import ChatOpenAI, OpenAIEmbeddings # Now this import should work
import openai
import faiss 
load_dotenv()

import os

openai.api_key = ''

In [12]:
from langchain.document_loaders import TextLoader

# 파일 경로 설정
input_file_path = 'KakaoTalk_20240815_1548_52_097_group_단비.txt'

# TextLoader를 사용하여 파일을 로드합니다.
loader = TextLoader(input_file_path)
datas = loader.load()

# # 로드된 문서를 출력합니다.
# for data in datas:
#     print(data)
datas

[Document(metadata={'source': 'KakaoTalk_20240815_1548_52_097_group_단비.txt'}, page_content='멸종보호단체 님과 카카오톡 대화\n저장한 날짜 : 2024-08-15 15:49:07\n\n--------------- 2024년 7월 15일 월요일 ---------------\n[배성욱(열정)] [오전 8:06] 사진\n[배성욱(열정)] [오전 8:06] 8시인데 벌써\n[김호준(열정)] [오전 8:07] ㅋㅋㅋㅋㅋㅋㅋ̆̎\n[옥지원(열정)] [오전 8:07] 와\n[김호준(열정)] [오전 8:07] 세번째중 ㄱㄱ\n[배성욱(열정)] [오전 8:07] 옆분단?\n[김호준(열정)] [오전 8:07] ㅇㅇ\n[옥지원(열정)] [오전 8:07] 나란히 쭉 앉을거?\n[배성욱(열정)] [오전 8:09] 어떻게 할까\n[배성욱(열정)] [오전 8:09] 일단\n[배성욱(열정)] [오전 8:09] 딴조들한테\n[배성욱(열정)] [오전 8:09] 일렬로 앉는다고 하긴했으\n[옥지원(열정)] [오전 8:09] 응응 얼른 갈게...\n[배성욱(열정)] [오전 8:30] 사진\n[배성욱(열정)] [오전 8:30] 엄 \n[배성욱(열정)] [오전 8:30] 네번째 이동?\n[김호준(열정)] [오전 8:31] 원래 자리로 가게되겠네요\n[옥지원(열정)] [오전 8:31] 헉\n[김호준(열정)] [오전 8:31] ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ\n[옥지원(열정)] [오전 8:32] ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ\n[옥지원(열정)] [오전 8:32] 아니 오빠만 일직 갈게 아니라 우리도 빨리가야하네..\n[김호준(열정)] [오전 8:32] ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ\n[배성욱(열정)] [오전 8:32] ㅋㅋㅋㅌㅌㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ\n[배성욱(열정)] [오전 8:32] 갸웃기네\n[배성욱(열정)] [오전 8:33] 원래자리 가?\n[옥지원(열정)] [오전 8:33] 다들 괜차나 맨뒤?\n[김현동] [오전 8:33] 음.. 첫줄은 부

In [13]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

splits = text_splitter.split_documents(datas)
splits

[Document(metadata={'source': 'KakaoTalk_20240815_1548_52_097_group_단비.txt'}, page_content='멸종보호단체 님과 카카오톡 대화\n저장한 날짜 : 2024-08-15 15:49:07'),
 Document(metadata={'source': 'KakaoTalk_20240815_1548_52_097_group_단비.txt'}, page_content='--------------- 2024년 7월 15일 월요일 ---------------\n[배성욱(열정)] [오전 8:06] 사진\n[배성욱(열정)] [오전 8:06] 8시인데 벌써\n[김호준(열정)] [오전 8:07] ㅋㅋㅋㅋㅋㅋㅋ̆̎\n[옥지원(열정)] [오전 8:07] 와\n[김호준(열정)] [오전 8:07] 세번째중 ㄱㄱ\n[배성욱(열정)] [오전 8:07] 옆분단?\n[김호준(열정)] [오전 8:07] ㅇㅇ\n[옥지원(열정)] [오전 8:07] 나란히 쭉 앉을거?\n[배성욱(열정)] [오전 8:09] 어떻게 할까\n[배성욱(열정)] [오전 8:09] 일단\n[배성욱(열정)] [오전 8:09] 딴조들한테\n[배성욱(열정)] [오전 8:09] 일렬로 앉는다고 하긴했으\n[옥지원(열정)] [오전 8:09] 응응 얼른 갈게...\n[배성욱(열정)] [오전 8:30] 사진\n[배성욱(열정)] [오전 8:30] 엄 \n[배성욱(열정)] [오전 8:30] 네번째 이동?\n[김호준(열정)] [오전 8:31] 원래 자리로 가게되겠네요\n[옥지원(열정)] [오전 8:31] 헉\n[김호준(열정)] [오전 8:31] ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ\n[옥지원(열정)] [오전 8:32] ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ\n[옥지원(열정)] [오전 8:32] 아니 오빠만 일직 갈게 아니라 우리도 빨리가야하네..\n[김호준(열정)] [오전 8:32] ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ\n[배성욱(열정)] [오전 8:32] ㅋㅋㅋㅌㅌㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ\n[배성욱(열정)] [오전 8:32] 

In [14]:
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings(openai_api_key=openai.api_key))

# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()
print(vectorstore)
print(retriever)

tags=['FAISS', 'OpenAIEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000028CF19F01D0>


In [15]:
prompt = hub.pull("rlm/rag-prompt",api_key=openai.api_key)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [16]:
llm = ChatOpenAI(model_name="gpt-4o", temperature=0,api_key=openai.api_key)


def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)


# 체인을 생성합니다.
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [25]:
result = rag_chain.invoke(
"""해당 대화에서 김현동(열정)의 언어습관을 분석해줘.  """
)  #문서에 대한 질의를 입력하고, 답변을 출력합니다.

In [26]:
result

'김현동의 언어습관은 질문을 통해 대화를 이끌어가는 경향이 있으며, 논리적이고 분석적인 접근을 선호합니다. 그는 대화 중에 다양한 관점을 제시하고, 문제의 원인과 해결책을 탐구하는 모습을 보입니다. 또한, 유머를 섞어 분위기를 부드럽게 만드는 특징도 있습니다.'

In [ ]:
result

'김현동의 언어습관은 간결하고 직설적이며, 자주 줄임말을 사용합니다. 또한, 친구를 격려하거나 농담을 섞어 대화를 이끌어가는 경향이 있습니다. 영어 단어를 간간이 섞어 쓰기도 합니다.'